# Preprocessing

In [1]:
# math and data packages
import pandas as pd
import numpy as np
import math

# charting and graphics
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# os and file types
import os
import sys
import datetime as dt
import json
import csv

# images and display
import base64, io, IPython
from PIL import Image as PILImage
from IPython.display import Markdown as md
from IPython.display import display, Math, Latex



## Import data

Fix any known formatting problems here

In [2]:
# make slugs for species and places

# function to make the species slugs
def to_slug(x):
    try: 
        int_data = x.split()
        data = int_data[:2]
        data = "-".join(data)
        data = data.lower()
    except:
        data = "none"
    return data

# chaux de fonds 5x5 kilometer = 550_215
# Biel = the other four of them.
# with Biel, just covers the general survey area. Make new list of unique values from each of the 4 biel lists.

cdf5x5 = pd.read_csv("resources/atlas5x5/Atlas5x5_550_215.csv", encoding = "utf-16")
cdf5x5.head()

,Taxon_Id,SISF,species,WS-Code,Eingeschleppt,Obs.Anz.,Jahre (min),Jahre (max),Unsicher (?)
0,1000000,100.0,Abies alba Mill.,Häufig,Indigen,31,1955,2021,NaN
1,1000070,300.0,Acer campestre L.,Häufig,Indigen,5,1955,2001,NaN
2,1000120,700.0,Acer platanoides L.,Häufig,Indigen,7,1955,2001,NaN
3,1000130,800.0,Acer pseudoplatanus L.,Häufig,Indigen,33,1955,2021,NaN
4,1000160,900.0,Aceras anthropophorum (L.) W. T. Aiton,Häufig,Indigen,1,1982,1999,NaN


## Determine wether or not a species was detected within a geographic limit

The territory is divided into different segments. Flora-helvitica and WS maps have different geographic bounds. Here the presence or not of a species within the confines of one of the different boundaries is determined.

### Key the species to the different maps it was identified in


## Format date column to ISO standard